# Lösen von Rekurrenzgleichungen

Viele Algorithmen basieren auf Rekursion. Für die Komplexitätsanalyse solcher Algorithmen ist ein Verfahren von Nöten, mit welchem rekursive Gleichungen (Rekurrenzgleichungen) gelöst werden können.

Ein typisches Beispiel ist die Fibonacci-Folge: 1, 1, 2, 3, 5, 8, 13, 21, 34, 55,...

Die Bildungsvorschrift der $n$-ten Fibonacci-Zahl kann folgendermaßen angegeben werden:

$$
\begin{align}
fib(0) & = 0 \\
fib(1) & = 1 \\
fib(n) & = fib(n - 1) + fib(n - 2) | n \geqslant 2
\end{align}
$$

Die $n$-te Fibonacci Zahl ist gleich der Summe der zwei vorhergehenden Zahlen.

In [221]:
def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fib(n-1) + fib(n-2)

print(list(map(fib, list(range(0, 20)))))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]


Der Zeitaufwand lässt sich ebenfalls durch eine rekursive Gleichung ausdrücken:

$$
\begin{align}
T(0) & = 1 \\
T(1) & = 1 \\
T(n) & = T(n - 1) + T(n - 2) | n \geqslant 2
\end{align}
$$

Nun ist es Ziel für $T(n)$ einen Ausdruck zu finden, der nicht von $T(i)$ abhängt.

## Raten und Einsetzen

Ein Lösungsmethode ist das __Intelligent guesswork__ - das geschickte Raten. Hierfür stellt man eine Wertetabelle für $T(n)$ auf und versucht daraus eine explizite Bildungsvorschrift zu erkennen.

Zu lösen sei folgende Rekursionsgleichung:

$$
\begin{align}
T(1) & = 1 \\
T(n) & = 3 \cdot T(\frac{n}{2}) + n
\end{align}
$$

$n$ sei hierbei eine Zweierpotenz, d.h. $n = 2^k$ mit $k \in \mathbb{N}$.

In [222]:
def T(n):
    if n == 1:
        return 1
    return 3 * T(n/2) + n


args = list(map(lambda n: 2**n, list(range(1, 8))))
print('n\tT(n)')
print('------------')
for n in args:
    print(str(n) + '\t' + str(int(T(n))))

n	T(n)
------------
2	5
4	19
8	65
16	211
32	665
64	2059
128	6305


Gibt man zusätzlich zu den Funktionswerten die Summendarstellungen an, ergibt sich folgende Wertetabelle:

|$n$|$T(n)$|
|---:|---:|
|$1$|$1$|
|$2$|$5=3 \cdot 1 + 2$|
|$4$|$19=3^2 \cdot 1 + 3 \cdot 2 + 2^2$|
|$8$|$65=3^3 \cdot 1 + 3^3 \cdot 2 + 3 \cdot 2^2 + 2^3$|
|$16$|$211=3^4 \cdot 1 + 3^3 \cdot 2 + 3^2 \cdot 2^2 + 3 \cdot 2^3 + 2^4$|
|$32$|$665=3^5 \cdot 1 + 3^4 \cdot 2 + 3^3 \cdot 2^2 + 3^2 \cdot 2^3 + 3 \cdot 2^4 + 2^5$|

Mit Hilfe dieser Summendarstellung lässt sich ein gewisses Muster erkennen, dadurch kann die Lösung "erraten" werden.

$$
\begin{align}
T(2^k) & = 3^k \cdot 2^0 + 3^{k-1} \cdot 2^1 + ... + 3^1 \cdot 2^{k-1} + 3^0 \cdot 2^k \\
 & = \sum_{i=0}^{k}(3^{k-i} \cdot 2^i) \\
 & = 3^k \sum_{i=0}^k(\frac{2}{3})^i \\
 & = 3^k \frac{1-(\frac{2}{3}^{k+1})}{1-\frac{2}{3}} \\
T(2^k) & = 3^{k+1} - 2^{k+1}
\end{align}
$$

Um eine Funktion $n \rightarrow T(n)$ zu erhalten, muss $k$ durch $\log_2 n$ ersetzt werden.

$$
\begin{align}
T(n) & = 3^{\log_2 (n) + 1} - 2^{\log_2 (n) + 1} \\
 & = 3^{\log_2 n} \cdot 3^1 - 2^{\log_2 n} \cdot 2^1 \\
 & = 3 \cdot 3^{\log_2 n} - 2 \cdot 2^{\log_2 n} \\
T(n) & = 3 \cdot n^{\log_2 3} - 2 \cdot n
\end{align}
$$

Um die asymptotische Aufwandsordnung anzugeben, kann $-2n$ und der Faktor $3$ vernachlässigt werden. Dies ergibt $\mathcal{O}(n^{\log_2 3})$.

## Iterationsmethode

Bei der __Iterationsmethode__ wird eine rekursive Vorschrift solange angewandt, bis man zu einem rekursionsfreien Ausdruck gelangt. Dies geschieht durch ständiges Einsetzen der rekursiven Funktionsaufrufe. Diese Expansion durch Selbstanwendung wird __Telescoping__ genannt.

Hat man eine rekursive Funktion $T(n)$ und setzt für $n$ einen konkreten Wert ein, so kann problemlos Telescoping angewandt werden, da in endlich vielen Schritten die Elementarfälle erreicht werden und ein rekursionsfreier Ausdruck entsteht. Möchte man aber eine rekursive Funktion $T(n)$ für ein allgemeines $n$ mit Hilfe der Iterationsmethode lösen, so ist ein mathematischer Zwischenschritt nötig.

Gegeben sei folgendes Beispiel $T(n)$:

$$
\begin{align}
T(1) & = 1 \\
T(n) & = 2 \cdot T(\frac{n}{4}) + n
\end{align}
$$

Die Gleichung wird nun schrittweise expandiert:

$$
\begin{align}
T(n) & = 2 \cdot T(\frac{n}{4}) + n \\
 & = 2 \cdot (2 \cdot T(\frac{n}{16}) + \frac{n}{4}) + n \\
 & = 4 \cdot T(\frac{n}{16}) + \frac{3}{2}n \\
 & = 4 \cdot (2 \cdot T(\frac{n}{64}) + \frac{n}{16}) + \frac{3}{2}n \\
 & = 8 \cdot T(\frac{n}{64}) + \frac{7}{4}n \\
 & = 8 \cdot (2 \cdot T(\frac{n}{256}) + \frac{n}{64}) + \frac{7}{4}n \\
 & = 16 \cdot T(\frac{n}{256}) + \frac{15}{8}n \\
\end{align}
$$

Es wird ein gewisses Muster ersichtlich, welches sich mit einer Variable $i$ ausdrücken lässt.

$$
\begin{align}
T(n) & = 2^i \cdot T(\frac{n}{4^i}) + \frac{2^i - 1}{2^{i-1}} \cdot n \\
\end{align}
$$

Nun muss $i$ so gewählt werden, dass aus $T(\frac{n}{4^i})$ ein rekursionsfreier Ausdruck ensteht, also der Elementarfall erreicht ist. Dies geschieht bei $T(1)$:

$$
\begin{align}
T(\frac{n}{4^i}) & = T(1) \\
\frac{n}{4^i} & = 1 \\
n & = 4^i \\
i & = \log_4 n
\end{align}
$$

Jetzt kann $\log_4 n$ für $i$ eingesetzt werden:

$$
\begin{align}
T(n) & = 2^{\log_4 n} \cdot T(\frac{n}{4^{\log_4 n}}) + \frac{2^{\log_4 n} - 1}{2^{\log_4 (n) - 1}} \cdot n \\
 & = n^{\log_4 2} \cdot T(1) + \frac{n^{\log_4 2} - 1}{\frac{2^{\log_4 n}}{2}} \cdot n \\
 & = n^{\log_4 2} \cdot T(1) + \frac{n^{\log_4 2} - 1}{\frac{n^{\log_4 2}}{2}} \cdot n \\
 & = n^{\frac{1}{2}} \cdot T(1) + \frac{n^{\frac{1}{2}} - 1}{\frac{n^{\frac{1}{2}}}{2}} \cdot n \\
 & = n^{\frac{1}{2}} + \frac{2n^{\frac{1}{2}} - 2}{n^{\frac{1}{2}}} \cdot n \\
 & = n^{\frac{1}{2}} + \frac{2n^{\frac{3}{2}} - 2n}{n^{\frac{1}{2}}} \\
 & = n^{\frac{1}{2}} + \frac{2n^{\frac{3}{2}} - 2n}{n^{\frac{1}{2}}} \\
 & = n^{\frac{1}{2}} + 2n - 2n^\frac{1}{2} \\
T(n) & = 2n - n^\frac{1}{2}
\end{align}
$$